In [67]:
import pandas as pd
import numpy as np

from statistics import mean

from sklearn.preprocessing import LabelEncoder

from sklearn.feature_selection import SelectKBest

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import classification_report

# Import dataset

In [68]:
compressed_final = pd.read_csv("../DataFormating/compressed_final.csv")
final = pd.read_csv("../DataFormating/final.csv")

# Setup `X`, `y` data for training / testing

In [69]:
compressed_final.columns[:10]

Index(['Stage', 'Home Team Name', 'Home Team Goals', 'Away Team Goals',
       'Away Team Name', 'Attendance', 'Half-time Home Goals',
       'Half-time Away Goals', 'Home Team Initials', 'Away Team Initials'],
      dtype='object')

In [70]:
X = compressed_final.drop(["Home Team Goals", "Away Team Goals", 
                           "Half-time Home Goals", "Half-time Away Goals", 
                           "Home Team Initials", "Away Team Initials"], axis=1)
y = []

In [71]:
for i in range(len(compressed_final)):
    home_team_goals = compressed_final.iloc[i]["Home Team Goals"]
    away_team_goals = compressed_final.iloc[i]["Away Team Goals"]
    
    if home_team_goals > away_team_goals:
        y.append(1)
    elif home_team_goals < away_team_goals:
        y.append(2)
    else:
        y.append(0)

In [72]:
# Test
assert len(X) == len(y)

### Feature Selection

In [73]:
X.columns[4:]

Index(['Player 1 Age Diff', 'Player 1 Overall Diff', 'Player 1 Potential Diff',
       'Player 1 Acceleration Diff', 'Player 1 Aggression Diff',
       'Player 1 Agility Diff', 'Player 1 Balance Diff',
       'Player 1 Ball control Diff', 'Player 1 Composure Diff',
       'Player 1 Crossing Diff',
       ...
       'Player 11 Shot power Diff', 'Player 11 Sliding tackle Diff',
       'Player 11 Sprint speed Diff', 'Player 11 Stamina Diff',
       'Player 11 Standing tackle Diff', 'Player 11 Strength Diff',
       'Player 11 Vision Diff', 'Player 11 Volleys Diff',
       'Mean Home Team Goals', 'Mean Away Team Goals'],
      dtype='object', length=409)

In [74]:
feature_names = [
    "Stage", "Home Team Name", "Away Team Name", "Attendance",
    "Overall", # "Potential",
    "Mean Home Team Goals", "Mean Away Team Goals"
]

COLUMNS = []

for column_name in X.columns:
    for feature_name in feature_names:
        if feature_name in column_name:
            COLUMNS.append(column_name)
            break


In [75]:
X = X[COLUMNS]

### Encode textual features from the `X` dataset

In [76]:
X["Stage"] = LabelEncoder().fit_transform(X["Stage"])
X["Home Team Name"] = LabelEncoder().fit_transform(X["Home Team Name"])
X["Away Team Name"] = LabelEncoder().fit_transform(X["Away Team Name"])

In [77]:
len(X.columns)

17

# Fast testing

In [78]:
def test_model(model):
    mean_train_acc = mean(cross_validate(model, X, y, cv=50)["train_score"]) 
    mean_test_acc = mean(cross_validate(model, X, y, cv=50)["test_score"])
    
    print("Train Accuracy: ", mean_train_acc)
    print("Test Accuracy: ", mean_test_acc)

### Random Forests

In [79]:
test_model(RandomForestClassifier(n_estimators=100))

/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 33 members, which is too few. The minimum number of members in any class cannot be less than n_splits=50.
  % (min_groups, self.n_splits)), Warning)
/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 33 members, which is too few. The minimum number of members in any class cannot be less than n_splits=50.
  % (min_groups, self.n_splits)), Warning)


Train Accuracy:  1.0
Test Accuracy:  0.7613333333333333


### Support Vector Machines

In [80]:
test_model(SVC(C=1.0, kernel="rbf", gamma="auto"))

/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 33 members, which is too few. The minimum number of members in any class cannot be less than n_splits=50.
  % (min_groups, self.n_splits)), Warning)
/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 33 members, which is too few. The minimum number of members in any class cannot be less than n_splits=50.
  % (min_groups, self.n_splits)), Warning)


Train Accuracy:  1.0
Test Accuracy:  0.4146666666666667


### Extremely Randomized Trees

In [81]:
test_model(ExtraTreesClassifier(n_estimators=100))

/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 33 members, which is too few. The minimum number of members in any class cannot be less than n_splits=50.
  % (min_groups, self.n_splits)), Warning)
/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/victor/Documents/WorldCup-Prediction/.venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 33 members, which is too few. The minimum number of members in any class cannot be less than n_splits=50.
  % (min_groups, self.n_splits)), Warning)


Train Accuracy:  1.0
Test Accuracy:  0.7596666666666667
